In [1]:
#필요한 라이브러리
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
plt.rc('font', size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style='white')
sns.set(style='whitegrid', color_codes=True)

C:\Users\ship2\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df=pd.read_csv('baking.csv',header=0)

In [3]:
#데이터 불러오기
data=df
data=data.dropna()
print(data.shape)
print(list(data.columns))

(45211, 17)
['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'Target']


In [4]:
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,Target
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [5]:
#교육 데이터 보기
data['education'].unique()

array(['tertiary', 'secondary', 'unknown', 'primary'], dtype=object)

In [6]:
#기본 교육 기간자들 하나의 항목으로 합치기
data['education']=np.where(data['education']=='basic.9y','Basic',data['education'])
data['education']=np.where(data['education']=='basic.6y','Basic',data['education'])
data['education']=np.where(data['education']=='basic.4y','Basic',data['education'])

In [7]:
data['education'].unique()

array(['tertiary', 'secondary', 'unknown', 'primary'], dtype=object)

In [8]:
#정기예금 가입자 수 보기
#data['y'].value_counts()

KeyError: 'y'

In [ ]:
#정기예금에 가입한 사람과 가입하지 않은 사람 보기
sns.countplot(x='y',data=data,palette='hls')
plt.show()
plt.savefig('count_plot')

In [ ]:
#정기예금 가입자 수 퍼센테이지 구하기
count_nonsub=len(data[data['y']==0])
count_sub=len(data[data['y']==1])
percent_nonsub=count_nonsub/(count_nonsub+count_sub)
print('percent of nonsubscribers',percent_nonsub*100)
percent_sub=count_sub/(count_nonsub+count_sub)
print('percent of subscribers',percent_sub*100)

#89:11 로 비 가입자 수가 8배 많다.

In [ ]:
#가입자와 비가입자의 평균 구해보기
data.groupby('y').mean()
# 3가지 특징 정리
# 1. 정기예금을 가입한 사람들은 평균나이가 가입하지 않은 사람들에 비해 높았다.
# 2. 가입한 사람이 마지막 연락기간까지 기간이 상대적으로 짧다.
# 3. 가입한 사람들에 대한 홍보가 더 적다. (campaign 수가 적다.)

In [ ]:
#이제 교육(education), 결혼여부(marital), 직업(job) 등 카테고리 별로 평균을 구할 것이다.

data.groupby('education').mean()

In [ ]:
data.groupby('marital').mean()

In [ ]:
data.groupby('job').mean()

In [ ]:
#이를 시각화 해봐서 비교하기
table=pd.crosstab(data.education,data.y)
table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)
plt.title('Purchase ratio sorted by edu')
plt.xlabel('Education')
plt.ylabel('Proportion of Customers')
plt.savefig('edu_vs_pur_stack')
#교육수준이 좋은 예측수단이 될듯

In [ ]:
table=pd.crosstab(data.marital,data.y)
table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)
plt.title('Purchase ratio sorted by marital')
plt.xlabel('Marital')
plt.ylabel('Proportion of Customers')
plt.savefig('marital_vs_pur_stack')

In [ ]:
%matplotlib inline
pd.crosstab(data.job,data.y).plot(kind='bar')
plt.title('Purchase Frequency for Job')
plt.xlabel('Job')
plt.ylabel('Frequency of Purchase')
plt.savefig('purchase_fre_job')

In [ ]:
#월별로 구분
pd.crosstab(data.month,data.y).plot(kind='bar')
plt.title('Purchase Frequency for Month')
plt.xlabel('Month')
plt.ylabel('Frequency of Purchase')
plt.savefig('pur_fre_month_bar')

In [ ]:
#연령대별로 구분
data.age.hist()
plt.title('Histogram of Age')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.savefig('hist_age')
#30-40대가 가장 많다.

In [ ]:
pd.crosstab(data.poutcome,data.y).plot(kind='bar')
plt.title('Purchase Frequency for Poutcome')
plt.xlabel('Poutcome')
plt.ylabel('Frequency of Purchase')
plt.savefig('pur_fre_pout_bar')

In [ ]:
cat_vars=['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']
for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(data[var], prefix=var)
    data1=data.join(cat_list)
    data=data1

In [ ]:
cat_vars=['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']
data_vars=data.columns.values.tolist()
to_keep=[i for i in data_vars if i not in cat_vars]

In [ ]:
data_final=data[to_keep]
data_final.columns.values

In [ ]:
data_final_vars=data_final.columns.values.tolist()
y=['y']
X=[i for i in data_final_vars if i not in y]

In [ ]:
# RFE를 통해 영향력이 큰 feature만 추려낸다.

from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

rfe = RFE(logreg, 18)
rfe = rfe.fit(data_final[X], data_final[y] )
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
#True 인 열만 추출하기
i = 0
cols = []
for e in rfe.support_: 
    if e == True:
        cols.append(X[i])
    i = i+1

In [ ]:
# RFE후 로지스틱 회귀를 실시한다.

X=data_final[cols]
y=data_final['y']
import statsmodels.api as sm
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

In [ ]:
#테스트 결과 예측하고 정확도를 보여준다.
y_pred = logreg.predict(X_test)

In [ ]:
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

In [ ]:
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
kfold = model_selection.KFold(n_splits=10, random_state=7)
modelCV = LogisticRegression()
scoring = 'accuracy'
results = model_selection.cross_val_score(modelCV, X_train, y_train, cv=kfold, scoring=scoring)
print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

In [ ]:
#10872+254개의 올바른 예측치를 and 1122+109개의 틀린 예측치를 보여준다.
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

In [ ]:
#전체적으로 81%로 변수를 통해 정기 예금 가입률에 대해 설명할 수 있다.
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
#마지막으로 모델 예측력을 파악하기 위해 roc곡선 그리기.
# 넓이는 0.6 이므로 유의미한 모형이라 할 수 있다.
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()